In [76]:
import cirq
import openfermion
import numpy as np
import datetime
Pi=3.1415


## info 
It is a note to reproduce [SciPost: SciPost Phys. 6, 029 (2019) - Efficient variational simulation of non-trivial quantum states](https://scipost.org/SciPostPhys.6.3.029) chapter 4.1.

I recommend to download me and run codes on your local environment which already installed cirq and openfermion.
It is also available in google colab, but it takes about 5 min to import libraries on every runtime.


## anzats
expZZ gate's phase coefficient is Pi/2 as default.
https://quantumai.google/cirq/experiments/qaoa/qaoa_ising#implementing_ugamma_c

In [77]:
class Anzats():
    def __init__(self, length, gamma, beta):

        # initialize circuit
        circuit = cirq.Circuit()
        qubits = cirq.LineQubit.range(length)

        for i in range(length):
            circuit.append(cirq.H(qubits[i]))

        # add gamma circuit
        for index in range(gamma.size):
            # add gates on 2n-1 and 2n
            for i in range(1, length, 2):
                circuit.append(
                    cirq.ZZ(qubits[i], qubits[(i+1)%length]) ** (gamma[index]*2/Pi)
                    )# 
                     
                # circuit.append(
                #     cirq.ZZPowGate(
                #         exponent=gamma[index]/Pi, global_shift=0.0).on(
                #             qubits[i], qubits[(i+1)%length]
                #     )
                # )
            # add gates on 2n and 2n+1
            for i in range(0, length, 2):
                circuit.append(
                    cirq.ZZ(qubits[i], qubits[(i+1)%length]) ** (gamma[index]*2/Pi)
                    )
                # circuit.append(
                #     cirq.ZZPowGate(
                #         exponent=gamma[index]/Pi, global_shift=0.0).on(
                #             qubits[i], qubits[(i+1)%length]
                #         )
                # )

            # add beta circuit
            for i in range(length):
                circuit.append(
                    cirq.XPowGate(
                        exponent=beta[index]*2/Pi, global_shift=0.0).on(
                        qubits[i])
                )

        self.circuit = circuit
        self.qubits = qubits
        self.gamma = gamma
        self.beta = beta




## expectation

In [78]:
def get_expectation_ghz(length, gamma, beta):
    anzats = Anzats(length, gamma, beta)
    circuit = anzats.circuit
    qubits = anzats.qubits
    vector = cirq.final_state_vector(circuit)

    value = 0 + 0j
    for i in range(length):
        circuit = anzats.circuit.copy()
        circuit.append(cirq.Z(qubits[i]))
        circuit.append(cirq.Z(qubits[(i+1)%length]))
        vector2 = cirq.final_state_vector(circuit)
        value -= np.dot(vector2.conj(), vector)
    return value


# avoid to input model arguments (length) to optimize_by_gradient_descent()
def get_expectation_ghz_l4(gamma: np.array, beta: np.array):
    return get_expectation_ghz(4, gamma, beta)

def get_expectation_ghz_l8(gamma: np.array, beta: np.array):
    return get_expectation_ghz(8, gamma, beta)

def get_expectation_ghz_l10(gamma: np.array, beta: np.array):
    return get_expectation_ghz(10, gamma, beta)

def get_expectation_ghz_l12(gamma: np.array, beta: np.array):
    return get_expectation_ghz(12, gamma, beta)

def get_expectation_ghz_l14(gamma: np.array, beta: np.array):
    return get_expectation_ghz(14, gamma, beta)

def get_expectation_ghz_l16(gamma: np.array, beta: np.array):
    return get_expectation_ghz(16, gamma, beta)


## optimization

In [79]:
def get_gradient(function, gamma: np.array, beta: np.array, delta_gamma, delta_beta, iter):
    grad_gamma = np.zeros_like(gamma)
    grad_beta  = np.zeros_like(beta)
    gamma_edge = gamma
    beta_edge  = beta
    # initial gamma, beta?
    
    if not (gamma.size == beta.size):
        return 1

    for index in range(gamma.size):
        center = gamma[index]
        gamma_edge[index] = gamma[index] - delta_gamma
        e1 = function(gamma_edge, beta)
        gamma_edge[index] = gamma[index] + delta_gamma
        e2 = function(gamma_edge, beta)
        grad_gamma[index] = (e2.real-e1.real)/(2*delta_gamma)
        gamma[index] = center

        center = beta[index]
        beta_edge[index] = beta[index] - delta_beta
        e1 = function(gamma, beta_edge)
        beta_edge[index] = beta[index] + delta_beta
        e2 = function(gamma, beta_edge)
        grad_beta[index] = (e2.real-e1.real)/(2*delta_beta)
        beta[index] = center
    
    return grad_gamma, grad_beta


def optimize_by_gradient_descent(function, initial_gamma: np.array, initial_beta: np.array, alpha, delta_gamma, delta_beta, iteration, Figure=True,filepath=""):
    gamma, beta = initial_gamma, initial_beta

    textlines = []
    textlines.append("iter, energy, gamma[0], beta[0],...")
    print("iter, energy, gamma[0], beta[0],...")

    for iter in range(int(iteration)):
        # it is complex for me to set get_gradient for two optical parameter_vector
        grad_gamma, grad_beta = get_gradient(function, gamma, beta, delta_gamma, delta_beta, iter)
        gamma -= alpha * grad_gamma
        beta  -= alpha * grad_beta
        energy = function(gamma, beta)

        record = [iter, energy]
        for index in range(gamma.size):
            record.append(gamma[index])
            record.append(beta[index])
        textlines.append(record)
        print(record)
    
    if len(filepath)>0:
        with open(filepath, mode='a') as f:
            for i, textline in enumerate(textlines):
                f.write("{}\n".format(textline))

    return gamma, beta

## case: length 4, static gamma and beta

In [80]:
length = 4
gamma = np.array([0.5, 0.4, 0.3, 0.2])
beta  = np.array([0.7, 0.5, 0.3, 0.1])

anzats = Anzats(length, gamma, beta)
circuit = anzats.circuit
print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))

value = get_expectation_ghz_l4(gamma, beta)
print("expectation on ghz: {}".format(value))

          ┌────────────────┐                        ┌────────────────┐                        ┌────────────────┐                        ┌────────────────┐
0: ───H────────────ZZ──────────ZZ─────────X^0.446────────────ZZ──────────ZZ─────────X^0.318────────────ZZ──────────ZZ─────────X^0.191────────────ZZ──────────ZZ─────────X^0.064───
                   │           │                             │           │                             │           │                             │           │
1: ───H────ZZ──────┼───────────ZZ^0.318───X^0.446────ZZ──────┼───────────ZZ^0.255───X^0.318────ZZ──────┼───────────ZZ^0.191───X^0.191────ZZ──────┼───────────ZZ^0.127───X^0.064───
           │       │                                 │       │                                 │       │                                 │       │
2: ───H────ZZ^0.318┼───────────ZZ─────────X^0.446────ZZ^0.255┼───────────ZZ─────────X^0.318────ZZ^0.191┼───────────ZZ─────────X^0.191────ZZ^0.127┼───────────ZZ─────────X^0.064───
  

## case: length 8, static gamma and beta (paper value)

In [81]:
length = 8
gamma = np.array([0.5297, 0.7243, 0.6151, 0.5243])
beta  = np.array([0.5243, 0.6151, 0.7243, 0.5297])

anzats = Anzats(length, gamma, beta)
circuit = anzats.circuit
print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))

value = get_expectation_ghz_l8(gamma, beta)
print("expectation on ghz: {}".format(value))

          ┌────────────────┐                        ┌──────────────────┐                         ┌────────────────┐                         ┌────────────────┐
0: ───H────────────ZZ──────────ZZ─────────X^(1/3)─────────────ZZ───────────ZZ──────────X^0.392────────────ZZ──────────ZZ─────────X^(6/13)────────────ZZ──────────ZZ─────────X^0.337───
                   │           │                              │            │                              │           │                              │           │
1: ───H────ZZ──────┼───────────ZZ^0.337───X^(1/3)────ZZ───────┼────────────ZZ^(6/13)───X^0.392────ZZ──────┼───────────ZZ^0.392───X^(6/13)────ZZ──────┼───────────ZZ^(1/3)───X^0.337───
           │       │                                 │        │                                   │       │                                  │       │
2: ───H────ZZ^0.337┼───────────ZZ─────────X^(1/3)────ZZ^(6/13)┼────────────ZZ──────────X^0.392────ZZ^0.392┼───────────ZZ─────────X^(6/13)────ZZ^(1/3)┼───────────

## case: length 4, optimize beta and gamma

In [82]:
length = 4

initial_gamma = np.array([0.1, 0.2])
initial_beta  = np.array([0.1, 0.2])

iteration = 10
alpha = 0.1
delta_gamma = 0.001
delta_beta  = 0.001

gamma, beta = optimize_by_gradient_descent(get_expectation_ghz_l4, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

iter, energy, gamma[0], beta[0],...
[0, (-2.032800257205963+0j), 0.3063244581222534, 0.18284151554107667, 0.3193299889564514, 0.421233069896698]
[1, (-2.2672213315963745+0j), 0.18308291435241697, 0.37888715267181394, 0.264675509929657, 0.3170411705970764]
[2, (-2.4059470295906067+0j), 0.297109580039978, 0.3057462930679321, 0.5140881657600402, 0.26239861249923707]
[3, (-2.458075523376465+0j), 0.28534660339355467, 0.6665481090545654, 0.37351955175399776, 0.3451447606086731]
[4, (-2.283917188644409+0j), 0.3647846937179565, 0.440503454208374, 0.790650737285614, 0.3785203814506531]
[5, (-2.0218955278396606+0j), 0.41740367412567136, 0.7523847579956054, 0.3654461026191711, 0.37891675233840943]
[6, (-1.5308891534805298+0j), 0.4420501947402954, 0.38337538242340086, 0.828693401813507, 0.39872337579727174]
[7, (-2.405406653881073+0j), 0.3683579921722412, 0.7417304277420044, 0.4061173319816589, 0.3555129885673523]
[8, (-1.8563450872898102+0j), 0.4010064363479614, 0.3915680408477783, 0.801105391979

## case: length 8, optimize beta and gamma

In [83]:
length = 8
p = 4
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta  = np.array([0.5 for i in range(p)])

iteration = 10
alpha = 0.01
delta_gamma = 0.001
delta_beta  = 0.001

gamma, beta = optimize_by_gradient_descent(get_expectation_ghz_l8, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

iter, energy, gamma[0], beta[0],...
[0, (-6.542201042175293+0j), 0.5004041194915771, 0.5322890281677246, 0.5268381834030151, 0.5117418169975281, 0.5117403268814087, 0.5268399715423584, 0.5322842597961426, 0.500406801700592]
[1, (-6.958579123020172+0j), 0.49947696924209595, 0.5556942820549011, 0.5474068522453308, 0.5257635116577148, 0.5257587432861328, 0.547411322593689, 0.5556877255439758, 0.4994480609893799]
[2, (-7.23870724439621+0j), 0.49781638383865356, 0.572079598903656, 0.5632727146148682, 0.5413784384727478, 0.5413709878921509, 0.5632816553115845, 0.5720584392547607, 0.4978014826774597]
[3, (-7.4482839703559875+0j), 0.4959537386894226, 0.5835305452346802, 0.5760892033576965, 0.5576880574226379, 0.5576856732368469, 0.5760620832443237, 0.583518922328949, 0.49594777822494507]
[4, (-7.612199306488037+0j), 0.4940893054008484, 0.5915351510047913, 0.5869101285934448, 0.5737068057060242, 0.573707103729248, 0.5868844985961914, 0.591528594493866, 0.4940773844718933]
[5, (-7.73878747224807

## case: length 8, optimize beta and gamma b

In [84]:
length = 8
p = 4
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta  = np.array([0.5 for i in range(p)])

iteration = 10
alpha = 0.1
delta_gamma = 0.001
delta_beta  = 0.001

gamma, beta = optimize_by_gradient_descent(get_expectation_ghz_l8, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

iter, energy, gamma[0], beta[0],...
[0, (-3.256655693054199+0j), 0.5040411949157715, 0.8228902816772461, 0.7683818340301514, 0.6174181699752808, 0.6174032688140869, 0.768399715423584, 0.8228425979614258, 0.5040680170059204]
[1, (-1.3659792095422745+0j), 0.52003014087677, 0.3772040009498596, 0.209436297416687, 0.4530524015426636, 0.4531954526901245, 0.20930218696594238, 0.3773545026779175, 0.5200137495994568]
[2, (-2.1984458565711975+0j), 0.23157793283462524, 0.5190995633602142, 0.16270998120307922, 0.46086880564689636, 0.46006637811660767, 0.1632397174835205, 0.518426775932312, 0.23158910870552063]
[3, (-2.5046935379505157+0j), 0.04661279916763306, 0.04132959246635437, 0.9091106355190277, 0.05352470278739929, 0.05527782440185547, 0.9066690802574158, 0.04145848751068115, 0.049882858991622925]
[4, (-4.318195939064026+0j), 0.4530524015426636, 0.03664317727088928, 1.310379534959793, 0.4920554459095001, 0.49062418937683105, 1.309776782989502, 0.036828696727752686, 0.4588005244731903]
[5, (-

In [85]:
gamma = np.array([0 for i in range(p)])
beta  = np.array([0 for i in range(p)])

anzats  = Anzats(length, gamma, beta)
circuit = anzats.circuit
print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

          ┌────────┐                ┌────────┐                ┌────────┐                ┌────────┐
0: ───H────────ZZ──────ZZ─────X^0────────ZZ──────ZZ─────X^0────────ZZ──────ZZ─────X^0────────ZZ──────ZZ─────X^0───
               │       │                 │       │                 │       │                 │       │
1: ───H────ZZ──┼───────ZZ^0───X^0────ZZ──┼───────ZZ^0───X^0────ZZ──┼───────ZZ^0───X^0────ZZ──┼───────ZZ^0───X^0───
           │   │                     │   │                     │   │                     │   │
2: ───H────ZZ^0┼───────ZZ─────X^0────ZZ^0┼───────ZZ─────X^0────ZZ^0┼───────ZZ─────X^0────ZZ^0┼───────ZZ─────X^0───
               │       │                 │       │                 │       │                 │       │
3: ───H────ZZ──┼───────ZZ^0───X^0────ZZ──┼───────ZZ^0───X^0────ZZ──┼───────ZZ^0───X^0────ZZ──┼───────ZZ^0───X^0───
           │   │                     │   │                     │   │                     │   │
4: ───H────ZZ^0┼───────ZZ─────X^0────ZZ^0┼───

2024-04-17T13:31

iteration = 100
alpha = 0.1
delta_gamma = 0.1
delta_beta  = 0.1

[-0.31614973 -0.31615938 -0.31615084 -0.31615441 -0.31615689 -0.31615102
 -0.31615095 -0.31614993] [4.62462144 4.62462131 4.62461898 4.62462046 4.62462403 4.62462154
 4.62462111 4.62461999]
[0.84503587 0.94503753 1.04503506 1.14503613 1.24503873 1.34503442
 1.44503386 1.54503706] [-2.72162034 -2.62161911 -2.52161576 -2.42161763 -2.32161937 -2.22161886
 -2.12161452 -2.02161869]

勾配の値が0以上\py/2以下になっていない

## case: length 16, static beta and gamma (paper value)

In [86]:
length = 16
p = 8
initial_gamma = np.array([0.5846,0.6105,0.7966,0.6373,0.7745,0.6152,0.7155,0.5796])
initial_beta  = np.array([0.5796,0.7155,0.6152,0.7745,0.6373,0.7966,0.6105,0.5846])

print(get_expectation_ghz_l16(initial_gamma, initial_beta))

# iteration = 100
# alpha = 0.1
# delta_gamma = 0.001
# delta_beta  = 0.001

# gamma, beta = optimize_by_gradient_descent(get_expectation_ghz_l16, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

(-15.999917387962341+0j)
[0 0 0 0] [0 0 0 0]


## playground

In [87]:
L = 10
circuit = cirq.Circuit()
qubits = cirq.LineQubit.range(L)

for i in range(1, L, 2):
    circuit.append(
        cirq.ZZPowGate(
            exponent=0.5, global_shift=0.0).on(
                qubits[i-1], qubits[i]
        )
    )


for i in range(2, L, 2):
    circuit.append(
        cirq.ZZPowGate(
            exponent=0.5, global_shift=0.0).on(
                qubits[i-1], qubits[i]
            )
    )


for i in range(L):
    circuit.append(
        cirq.XPowGate(
            exponent=0.5, global_shift=0.0).on(
            qubits[i])
    )


# for i in range(1, L, 2):
#     circuit.append(cirq.Z(qubits[i-1]))
#     circuit.append(cirq.Z(qubits[i]))


# for i in range(2, L, 2):
#     circuit.append(cirq.Y(qubits[i-1]))
#     circuit.append(cirq.Y(qubits[i]))


# for i in range(L):
#     circuit.append(cirq.X(qubits[i]))



print(circuit)


0: ───ZZ───────X^0.5────────────
      │
1: ───ZZ^0.5───ZZ───────X^0.5───
               │
2: ───ZZ───────ZZ^0.5───X^0.5───
      │
3: ───ZZ^0.5───ZZ───────X^0.5───
               │
4: ───ZZ───────ZZ^0.5───X^0.5───
      │
5: ───ZZ^0.5───ZZ───────X^0.5───
               │
6: ───ZZ───────ZZ^0.5───X^0.5───
      │
7: ───ZZ^0.5───ZZ───────X^0.5───
               │
8: ───ZZ───────ZZ^0.5───X^0.5───
      │
9: ───ZZ^0.5───X^0.5────────────


In [88]:
value = 0 + 0j

# circuit2 = circuit
# anzats2 = Anzats(length, gamma, beta)
# circuit2 = anzats2.circuit
# qubits = anzats2.qubits
# circuit2.append(
#         cirq.ZZPowGate(
#             exponent=1, global_shift=0.0).on(
#                 qubits[1], qubits[2]
#         ))
# print(circuit2)
# result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
# vector2 = cirq.final_state_vector(circuit2)
# value -= np.dot(vector2.conjugate(), vector)

# print(value)

# # for i in range(length-1):
#     circuit2 = circuit.append(cirq.ZZ.on(
#         qubits[i], qubits[i+1]
#     ))
#     result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
#     vector2 = cirq.final_state_vector(circuit2)
#     value -= np.dot(vector2.conjugate(), vector)
# else:
#     circuit2 = circuit.append(cirq.ZZ.on(
#         qubits[length-1], qubits[0]
#     ))
#     result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
#     vector2 = cirq.final_state_vector(circuit2)
#     value -= np.dot(vector2.conjugate(), vector)

In [89]:
a = np.array([1,3])
a[1]

3

In [90]:
print(vector)

[0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j 0.06249999+0.j
 0.06249999+0.j 0.062499

In [91]:
def function_norm(gamma, beta):
    return np.linalg.norm(gamma)**2 + np.linalg.norm(beta)**2

p = 1
initial_gamma = np.array([-3.0 for i in range(p)])
initial_beta  = np.array([4.0 for i in range(p)])

alpha = 0.1
delta_gamma = 0.0001
delta_beta = 0.0001
iteration = 100

final_gamma, final_beta = optimize_by_gradient_descent(function_norm, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration, Figure=True)

print(final_gamma, final_beta)

iter, energy, gamma[0], beta[0],...
[0, 20.250009000048443, -2.6999950000002855, 3.6000049999995696]
[1, 16.402515390162296, -2.4299904999994766, 3.2400094999975835]
[2, 13.286044756250423, -2.186986449998926, 2.9160135499979276]
[3, 10.761702813850011, -1.9682828049986085, 2.6244171949971715]
[4, 8.71698518447511, -1.771449524498145, 2.3619804754972904]
[5, 7.060763314251251, -1.5942995720487048, 2.1257874279467757]
[6, 5.71922306798135, -1.4348646148437645, 1.9132136851514012]
[7, 4.632574990255437, -1.2913731533593626, 1.721897316635964]
[8, 3.7523896168740816, -1.1622308380235786, 1.5497125849721591]
[9, 3.039439077053899, -1.0460027542212842, 1.3947463264749569]
[10, 2.461948791156269, -0.9413974787991748, 1.2552766938276303]
[11, 1.9941813457998523, -0.8472527309192319, 1.1297540244450364]
[12, 1.6152894326596015, -0.7625224578273722, 1.0167836220005242]
[13, 1.3083867288548008, -0.6862652120446207, 0.9151102598005076]
[14, 1.0597953100277273, -0.617633690840111, 0.82360423382043

In [92]:
def function_norm(gamma, beta):
    return np.linalg.norm(gamma)**2 + np.linalg.norm(beta)**2

initial_gamma = np.array([3.0, 2.0])
initial_beta = np.array([4.0, 5.0])

final_gamma, final_beta = optimize_by_gradient_descent(
    function=function_norm,
    initial_gamma=initial_gamma,
    initial_beta=initial_beta,
    alpha=0.1,
    delta_gamma=0.001,
    delta_beta=0.001,
    iteration=100,
    Figure=True)
print(final_gamma, final_beta)
print(function_norm(gamma, beta))

iter, energy, gamma[0], beta[0],...
[0, 43.741260009996445, 2.7000499999998624, 3.6000500000000955, 1.8000499999999846, 4.500049999999618]
[1, 35.43155463610003, 2.4300949999999517, 3.2400950000003945, 1.6200949999998642, 4.0500949999997715]
[2, 28.700579899441102, 2.1871354999997124, 2.9161355000003795, 1.4581354999997558, 3.6451354999999808]
[3, 23.248388317329418, 1.9684719499996035, 2.6245719500004725, 1.3123719499998003, 3.2806719500002757]
[4, 18.83202129493629, 1.7716747549995944, 2.3621647550002365, 1.1811847549998404, 2.9526547549999904]
[5, 15.254681350009095, 1.5945572794996394, 2.1259982795000596, 1.0631162794997522, 2.657439279499947]
[6, 12.35696160350955, 1.4351515515496267, 1.9134484515502557, 0.9568546515497083, 2.391745351550174]
[7, 10.00974165684386, 1.2916863963946952, 1.7221536063953344, 0.8612191863947665, 2.1526208163951743]
[8, 8.108433243244505, 1.162567756755248, 1.5499882457558698, 0.7751472677553366, 1.9374087347556923]
[9, 6.568319197107911, 1.046360981079

In [93]:
length = 6
gamma = np.array([0.1, 0.1])
beta = np.array([0.1, 0.1])

anzats = Anzats(length, gamma, beta)
circuit = anzats.circuit
qubits = anzats.qubits
vector = cirq.final_state_vector(circuit)

ham_nd = np.zeros((length, length), dtype=np.complex128)

print(cirq.Z(qubits[-1]))
for i in range(length-1):
    circuit_ZiZj = cirq.Circuit()
    circuit_ZiZj.append(cirq.Z(qubits[i]))
    circuit_ZiZj.append(cirq.Z(qubits[i+1]))
    ham_nd += cirq.unitary(circuit_ZiZj)
else:
    circuit_ZiZj = cirq.Circuit()
    circuit_ZiZj.append(cirq.Z(qubits[length-1]))
    circuit_ZiZj.append(cirq.Z(qubits[0]))
    ham_nd += cirq.unitary(circuit_ZiZj)
    
print(ham_nd)


# def get_expectation_ZiZj(length, beta, gamma, qubit_ZZ_on_i, qubit_ZZ_on_j, print_circuit=False):
#     """
#     controls of append on circuit are accumulated.
#     we have to make circuits on each hamiltonian terms.
#     """
#     anzats = Anzats(length, gamma, beta)
#     circuit = anzats.circuit
#     qubits = anzats.qubits
#     vector = cirq.final_state_vector(circuit)

#     anzats = Anzats(length, gamma, beta)
#     circuit = anzats.circuit
#     qubits = anzats.qubits
#     circuit.append(cirq.Z(qubits[qubit_ZZ_on_i]))
#     circuit.append(cirq.Z(qubits[qubit_ZZ_on_j]))
#     if print_circuit:
#         print(circuit)
#     simulator = cirq.Simulator()
#     # result2 = simulator.simulate(circuit, qubit_order=qubits)
#     vector2 = cirq.final_state_vector(circuit)
#     return np.dot(vector2.conjugate(), vector)


# def get_expectation_ghz_l4(gamma: np.array, beta: np.array):
#     length = 4
#     value = 0 + 0j
#     for i in range(length-1):
#         value -= get_expectation_ZiZj(length, gamma, beta, i, i+1)
#     else:
#         value -= get_expectation_ZiZj(length, gamma, beta, length-1, 0)
#     return value


# def get_expectation_ghz_l8(gamma: np.array, beta: np.array):
#     length = 8
#     value = 0 + 0j
#     for i in range(length-1):
#         value -= get_expectation_ZiZj(length, gamma, beta, i, i+1)
#     else:
#         value -= get_expectation_ZiZj(length, gamma, beta, length-1, 0)
#     return value


Z(q(5))


ValueError: operands could not be broadcast together with shapes (6,6) (4,4) (6,6) 